In [ ]:
# ! pip install pdfplumber langdetect pandas
# ! pip install fasttext



In [ ]:
# ! pip install scikit-learn

In [1]:
pdf_file = pdf_path="/Users/manishmaurya/Documents/Projects_galaxy/Mismatch language/47907491_BG_Spn _Cont.pdf"

# Example usage
pdf_path = pdf_file


In [2]:
# import fitz  # PyMuPDF
# from langdetect import detect
# from collections import Counter
# import pandas as pd
# import re

# # Function to validate a paragraph based on length and punctuation
# def is_valid_paragraph(text):
#     return (
#         len(text.split()) >= 8 and  # Minimum of 8 words
#         any(p in text for p in ['.', ':', ';', '!', '?'])  # End with punctuation
#     )

# # Function to handle multi-column layout
# def extract_text_by_columns(page, column_split=300):
#     blocks = page.get_text("blocks")
#     left_col, right_col = [], []

#     for b in blocks:
#         x0, y0, x1, y1, text, *_ = b
#         if x0 < column_split:
#             left_col.append((y0, text))
#         else:
#             right_col.append((y0, text))

#     left_col.sort()
#     right_col.sort()
#     combined_text = '\n'.join([t for _, t in left_col + right_col])
#     return combined_text

# # Extract paragraphs from the PDF
# def extract_paragraphs_from_pdf(pdf_path, use_columns=True, column_split=300):
#     doc = fitz.open(pdf_path)
#     paragraphs = []

#     for page_num, page in enumerate(doc, start=1):
#         text = extract_text_by_columns(page, column_split) if use_columns else page.get_text("text")
#         lines = [line.strip() for line in text.split('\n') if line.strip()]

#         para_buffer = ""
#         para_num = 0

#         for line in lines:
#             para_buffer += ' ' + line.strip()

#             if re.search(r'[.?!:;]$', line.strip()) or len(line.strip()) < 40:
#                 if is_valid_paragraph(para_buffer):
#                     para_num += 1
#                     paragraphs.append({
#                         'page': page_num,
#                         'paragraph_number': para_num,
#                         'text': para_buffer.strip(),
#                         'word_count': len(para_buffer.strip().split())
#                     })
#                 para_buffer = ""

#         if is_valid_paragraph(para_buffer):
#             para_num += 1
#             paragraphs.append({
#                 'page': page_num,
#                 'paragraph_number': para_num,
#                 'text': para_buffer.strip(),
#                 'word_count': len(para_buffer.strip().split())
#             })

#     return paragraphs

# # Detect language of each paragraph
# def detect_languages(paragraphs):
#     lang_results = []
#     for p in paragraphs:
#         try:
#             lang = detect(p['text'])
#         except:
#             lang = "unknown"
#         p['language'] = lang
#         lang_results.append(lang)
#     return paragraphs, lang_results

# # Identify the major language and extract foreign paragraphs
# def find_foreign_paragraphs(paragraphs, lang_results):
#     lang_count = Counter(lang_results)
#     major_language = lang_count.most_common(1)[0][0]

#     # Filter paragraphs that are not in the major language
#     foreign_paragraphs = [
#         p for p in paragraphs if p['language'] != major_language and p['language'] != "unknown"
#     ]

#     return major_language, foreign_paragraphs

# # Main function to analyze PDF language
# def analyze_pdf_language(pdf_path, use_columns=True, column_split=300):
#     paragraphs = extract_paragraphs_from_pdf(pdf_path, use_columns, column_split)
#     paragraphs, lang_results = detect_languages(paragraphs)
#     major_language, foreign_paragraphs = find_foreign_paragraphs(paragraphs, lang_results)

#     print(f"\n✅ Major language detected: {major_language}")
#     print(f"🚨 Foreign paragraphs found: {len(foreign_paragraphs)}")

#     # Create DataFrame for foreign paragraphs and all paragraphs
#     df_foreign = pd.DataFrame(foreign_paragraphs)
#     df_all = pd.DataFrame(paragraphs)

#     return major_language, df_foreign, df_all

# # Example Usage

# major_language, df_foreign, df_all = analyze_pdf_language(pdf_path)

# # Show the first few rows of the foreign paragraphs DataFrame
# # print(df_foreign.head())

# # Optionally, save the foreign language paragraphs DataFrame to CSV
# # df_foreign.to_csv('foreign_paragraphs.csv', index=False)


In [3]:
# Example usage
# pdf_path = pdf_file
# =======================
# 🚀 Example usage
# =======================

# pdf_path = "your_manual.pdf"

# # Enable column detection (for 2-column layout PDFs)
# major_lang, foreign_df, all_df = analyze_pdf_language(pdf_path, use_columns=True, column_split=300)

# # # Save to CSV
# # foreign_df.to_csv("foreign_language_paragraphs.csv", index=False)
# # all_df.to_csv("all_paragraphs_with_wordcount.csv", index=False)


In [ ]:
import fitz  # PyMuPDF
from langdetect import detect
from collections import Counter
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Function to validate a paragraph based on length and punctuation
def is_valid_paragraph(text):
    return (
        len(text.split()) >= 8 and  # Minimum of 8 words
        any(p in text for p in ['.', ':', ';', '!', '?'])  # End with punctuation
    )

# Function to handle multi-column layout
def extract_text_by_columns(page, column_split=300):
    blocks = page.get_text("blocks")
    left_col, right_col = [], []

    for b in blocks:
        x0, y0, x1, y1, text, *_ = b
        if x0 < column_split:
            left_col.append((y0, text))
        else:
            right_col.append((y0, text))

    left_col.sort()
    right_col.sort()
    combined_text = '\n'.join([t for _, t in left_col + right_col])
    return combined_text

# Extract paragraphs from the PDF
def extract_paragraphs_from_pdf(pdf_path, use_columns=True, column_split=300):
    doc = fitz.open(pdf_path)
    paragraphs = []

    for page_num, page in enumerate(doc, start=1):
        text = extract_text_by_columns(page, column_split) if use_columns else page.get_text("text")
        lines = [line.strip() for line in text.split('\n') if line.strip()]

        para_buffer = ""
        para_num = 0

        for line in lines:
            para_buffer += ' ' + line.strip()

            if re.search(r'[.?!:;]$', line.strip()) or len(line.strip()) < 40:
                if is_valid_paragraph(para_buffer):
                    para_num += 1
                    paragraphs.append({
                        'page': page_num,
                        'paragraph_number': para_num,
                        'text': para_buffer.strip(),
                        'word_count': len(para_buffer.strip().split())
                    })
                para_buffer = ""

        if is_valid_paragraph(para_buffer):
            para_num += 1
            paragraphs.append({
                'page': page_num,
                'paragraph_number': para_num,
                'text': para_buffer.strip(),
                'word_count': len(para_buffer.strip().split())
            })

    return paragraphs

# Detect language of each paragraph
def detect_languages(paragraphs):
    lang_results = []
    for p in paragraphs:
        try:
            lang = detect(p['text'])
        except:
            lang = "unknown"
        p['language'] = lang
        lang_results.append(lang)
    return paragraphs, lang_results

# Identify the major language and extract foreign paragraphs
def find_foreign_paragraphs(paragraphs, lang_results):
    lang_count = Counter(lang_results)
    major_language = lang_count.most_common(1)[0][0]

    # Filter paragraphs that are not in the major language
    foreign_paragraphs = [
        p for p in paragraphs if p['language'] != major_language and p['language'] != "unknown"
    ]

    return major_language, foreign_paragraphs

# Main function to analyze PDF language
def analyze_pdf_language(pdf_path, use_columns=True, column_split=300):
    paragraphs = extract_paragraphs_from_pdf(pdf_path, use_columns, column_split)
    paragraphs, lang_results = detect_languages(paragraphs)
    major_language, foreign_paragraphs = find_foreign_paragraphs(paragraphs, lang_results)

    print(f"\n✅ Major language detected: {major_language}")
    print(f"🚨 Foreign paragraphs found: {len(foreign_paragraphs)}")

    # Create DataFrame for foreign paragraphs and all paragraphs
    df_foreign = pd.DataFrame(foreign_paragraphs)
    df_all = pd.DataFrame(paragraphs)

    return major_language, df_foreign, df_all, lang_results

# Function to perform clustering based on language similarity
def cluster_by_language(df_all, num_clusters=4):
    # Use the 'language' column to create a similarity score for clustering
    lang_count = df_all['language'].value_counts()
    print(f"\nLanguage Distribution:\n{lang_count}")

    # Vectorizing the language column directly to find patterns based on language
    lang_vectorizer = TfidfVectorizer(stop_words='english')
    X_lang = lang_vectorizer.fit_transform(df_all['language'].fillna("unknown"))  # Fill NaNs if any

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    df_all['cluster'] = kmeans.fit_predict(X_lang)

    return df_all, kmeans




In [ ]:
# Example Usage

major_language, df_foreign, df_all, lang_results = analyze_pdf_language(pdf_path)

# Perform clustering based on language similarity
df_all_clustered, kmeans_model = cluster_by_language(df_all, num_clusters=4)

# Show the first few rows of the clustered DataFrame
print(df_all_clustered.head())

# Optionally, save the clustered DataFrame to CSV
# df_all_clustered.to_csv('clustered_paragraphs_by_language.csv', index=False)

In [ ]:
df_all_clustered.groupby('cluster')['language'].value_counts()

In [11]:
import fitz  # PyMuPDF
from langdetect import detect
from collections import Counter
import pandas as pd
import re

def is_valid_paragraph(text):
    """Heuristic to validate a meaningful paragraph."""
    word_count = len(text.split())
    has_punctuation = any(p in text for p in ['.', ':', ';', '!', '?'])
    not_code_like = not re.match(r'^\w{2,10}[-\d]*$', text.strip())  # avoid codes
    not_part_number = not re.search(r'\b(part\s*no|ref|code|item)\b', text.lower())
    not_dotted_line = not re.match(r'^.*\.{4,}.*$', text)  # remove lines with many dots (TOC-style)
    return word_count >= 8 and has_punctuation and not_code_like and not_part_number and not_dotted_line

def extract_text_by_columns(page, column_split=300):
    """Extracts text block-wise preserving column layout."""
    blocks = page.get_text("blocks")
    left_col, right_col = [], []

    for b in blocks:
        x0, y0, x1, y1, text, *_ = b
        if x0 < column_split:
            left_col.append((y0, text))
        else:
            right_col.append((y0, text))

    left_col.sort()
    right_col.sort()
    combined_text = '\n'.join([t for _, t in left_col + right_col])
    return combined_text

def clean_line(line):
    """Basic line hygiene."""
    line = line.strip()
    if not line or line.count('.') > 10:  # eliminate TOC-style lines
        return ''
    if re.fullmatch(r'[-–—_\s\d.]+', line):  # dashes, dots, whitespace only
        return ''
    return line

def extract_paragraphs_from_pdf(pdf_path, use_columns=True, column_split=300):
    doc = fitz.open(pdf_path)
    paragraphs = []

    for page_num, page in enumerate(doc, start=1):
        text = extract_text_by_columns(page, column_split) if use_columns else page.get_text("text")
        lines = [clean_line(line) for line in text.split('\n')]
        lines = [line for line in lines if line]

        para_buffer = ""
        para_num = 0

        for line in lines:
            if para_buffer:
                para_buffer += ' ' + line.strip()
            else:
                para_buffer = line.strip()

            if re.search(r'[.?!:;]$', line.strip()) or len(line.strip()) < 40:
                if is_valid_paragraph(para_buffer):
                    para_num += 1
                    paragraphs.append({
                        'page': page_num,
                        'paragraph_number': para_num,
                        'text': para_buffer.strip(),
                        'word_count': len(para_buffer.strip().split())
                    })
                para_buffer = ""

        if is_valid_paragraph(para_buffer):
            para_num += 1
            paragraphs.append({
                'page': page_num,
                'paragraph_number': para_num,
                'text': para_buffer.strip(),
                'word_count': len(para_buffer.strip().split())
            })

    return paragraphs

def detect_languages(paragraphs):
    lang_results = []
    for p in paragraphs:
        try:
            lang = detect(p['text'])
        except:
            lang = "unknown"
        p['language'] = lang
        lang_results.append(lang)
    return paragraphs, lang_results

def find_foreign_paragraphs(paragraphs, lang_results):
    lang_count = Counter(lang_results)
    major_language = lang_count.most_common(1)[0][0]

    foreign_paragraphs = [
        p for p in paragraphs if p['language'] != major_language and p['language'] != "unknown"
    ]

    return major_language, foreign_paragraphs

def analyze_pdf_language(pdf_path, use_columns=True, column_split=300):
    paragraphs = extract_paragraphs_from_pdf(pdf_path, use_columns, column_split)
    paragraphs, lang_results = detect_languages(paragraphs)
    major_language, foreign_paragraphs = find_foreign_paragraphs(paragraphs, lang_results)

    print(f"\n✅ Major language detected: {major_language}")
    print(f"🚨 Foreign paragraphs found: {len(foreign_paragraphs)}")

    df_foreign = pd.DataFrame(foreign_paragraphs)
    df_all = pd.DataFrame(paragraphs)
    return major_language, df_foreign, df_all


In [12]:
major_language, df_foreign, df_all=analyze_pdf_language(pdf_path)



✅ Major language detected: bg
🚨 Foreign paragraphs found: 8
